<img src="data/images/lecture-notebook-header.png" />

# Dimensionality Reduction: IRIS Dataset

Dimensionality reduction in data mining refers to the process of reducing the number of variables or features in a dataset while preserving its essential information. In data mining, datasets often contain a large number of variables, which can make analysis and interpretation challenging. Dimensionality reduction techniques aim to address this issue by transforming the data into a lower-dimensional space.

The primary goal of dimensionality reduction is to simplify the dataset by eliminating irrelevant or redundant features, noise, or highly correlated variables. By reducing the dimensionality, it becomes easier to visualize the data, explore relationships between variables, and build more efficient and accurate machine learning models. Moreover, dimensionality reduction can help alleviate the curse of dimensionality, which refers to the challenges and complexities that arise when working with high-dimensional data.

There are two main approaches to dimensionality reduction:

* **Feature Selection:** This approach involves selecting a subset of the original features while discarding the rest. The selection is typically based on criteria such as relevance to the target variable, statistical measures, or domain knowledge. Feature selection methods aim to identify the most informative variables that contribute significantly to the data patterns and predictive performance.

* **Feature Extraction:** This approach involves transforming the original features into a new set of variables, known as feature extraction or projection. Feature extraction methods create new features that are combinations or projections of the original ones, aiming to retain as much relevant information as possible while reducing dimensionality. Techniques such as Principal Component Analysis (PCA) and t-SNE (t-Distributed Stochastic Neighbor Embedding) are commonly used for feature extraction.

Both feature selection and feature extraction techniques have their own strengths and limitations. The choice of dimensionality reduction method depends on the specific characteristics of the dataset, the objectives of the analysis, and the algorithms or models to be employed downstream.

This notebooks introduces three very popular techniques for feature extraction: 

* Principal Component Analysis (PCA) 
* Linear Discriminant Analysis (LDA)
* t-distributed Stochastic Neighbor Embedding (t-SNE)

We use a very simple dataset with only 4 input features to go through the basic steps and without the need to consider any performance issues.

## Setting up the Notebook

### Specify how Plots Get Rendered

In [ ]:
%matplotlib inline

### Make all Required Imports

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_digits

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import matplotlib.cm as cm

---

## Load and Prepare Dataset (IRIS)

We use the [IRIS dataset](https://archive.ics.uci.edu/ml/datasets/iris) as a small but real-world dataset to illustrate the use of a KNN classifier. The Iris dataset is one of the most well-known and commonly used datasets in machine learning for classification tasks. It is named after the iris flower and was introduced by the statistician and biologist Ronald Fisher in 1936. The dataset is often used as a beginner's dataset for learning classification algorithms.

The Iris dataset consists of measurements of four features (attributes) of iris flowers, namely:

* Sepal Length (in centimeters)
* Sepal Width (in centimeters)
* Petal Length (in centimeters)
* Petal Width (in centimeters)

Based on these features, the dataset assigns each instance (row) to one of three classes of iris flowers: *Setosa*, *Versicolor*, and *Virginica*. The dataset contains 150 instances in total, with 50 instances per class. It is a balanced dataset, meaning that each class has an equal number of instances.

### Load Dataset from File

In [ ]:
df = pd.read_csv('data/datasets/iris/iris.csv')

# Encode string labels to numeric labels (we need this for LDA)
codes = {'setosa': 0, 'virginica': 1, 'versicolor': 2}
df['species'] = df['species'].map(codes)

df.head()

### Convert Dataframe to Numpy Arrays

As usual, the implementations for the dimensionality reduction techniques of `scikit-learn` expect `numpy` arrays as input

In [ ]:
X = df[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].to_numpy()
y = df[['species']].to_numpy().squeeze()

---------------------------

## Principal Component Analysis (PCA)

Principal Component Analysis (PCA) is a widely used dimensionality reduction technique in data mining and machine learning. It aims to transform a dataset consisting of possibly correlated variables into a new set of uncorrelated variables called principal components. These components are linear combinations of the original variables and capture the maximum amount of variation present in the data.

The main idea behind PCA is to project the data onto a lower-dimensional space while preserving as much information as possible. The first principal component accounts for the largest amount of variation in the data, the second component captures the second largest amount of variation, and so on. Each principal component is orthogonal to the others, meaning they are uncorrelated.

The steps involved in PCA are as follows:

* **Standardize the data:** If the variables in the dataset are measured on different scales, it is important to standardize them (subtracting the mean and dividing by the standard deviation) to ensure they contribute equally during the analysis.

* **Compute the covariance matrix:** The covariance matrix is calculated based on the standardized data. It represents the relationships between variables and indicates their degree of linear association.

* **Compute eigenvectors and eigenvalues:** The eigenvectors and eigenvalues are derived from the covariance matrix. Eigenvectors represent the directions or axes in the original feature space, while eigenvalues quantify the amount of variance explained by each eigenvector.

* **Select principal components:** The eigenvectors are ranked according to their corresponding eigenvalues, and the top-k eigenvectors with the highest eigenvalues are selected as the principal components. The number of principal components chosen depends on the desired level of dimensionality reduction.

* **Project the data:** The selected principal components are used as transformation matrices to project the original data onto the lower-dimensional space. This results in a new dataset consisting of the transformed variables, which are the principal components.

PCA can be beneficial for several reasons. It reduces the dimensionality of the data, making it easier to visualize and analyze. It can also help in removing noise and redundancy, focusing on the most important patterns and relationships. Furthermore, PCA can be used as a preprocessing step to improve the performance of machine learning algorithms by reducing the computational complexity and potential overfitting caused by high-dimensional data.


### Reducing from 4 to 2 dimensions

We first use PCA to reduce the 4 original features down to 2 new features to allow for easily plotting the results.

In [ ]:
pca_2d = PCA(n_components=2).fit(X)

X_pca_2d = pca_2d.transform(X)

As `X_pca_2d` now contains the 150 data sample, but now each with only two features we use a scatter plot for illustration.

In [ ]:
plt.figure()
plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap=cm.tab10, s=25)
plt.tick_params(labelsize=14)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

One can see that the 3 different classes are reasonably well separated. That indicates that there are indeed linear relationships between the original 4 input features. To further quantify this, we can look at the explained variances for the two 2 features. The explained variances are available in `PCA` after fitting the data.

In [ ]:
print('Explained variances (normalized): {}'.format(pca_2d.explained_variance_ratio_))

print('Overall explained variance (normalized): {}'.format(sum(pca_2d.explained_variance_ratio_)))

For a better presentation, we can also visualize the explained variances using a, in this example, box plot. We define a method `plot_explained_variances()` as we can reuse it for LDA.

In [ ]:
def plot_explained_variances(explained_variance_ratio):
    plt.figure()
    plt.tick_params(labelsize=14)
    plt.xlabel('x', fontsize=18)
    plt.ylabel('y', fontsize=18)
    plt.ylabel('percentage of variance (%)')
    plt.ylim(0, 110)
    x = np.arange(explained_variance_ratio.shape[0])
    labels = ['PC{}'.format(i+1) for i in x ]
    plt.bar(x, explained_variance_ratio*100, tick_label=labels)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_explained_variances(pca_2d.explained_variance_ratio_)

Looking and the explained variance, we can see that the first 2 principal components explain over 97.7% of the variances. This means that the dimensionality reduction from 4 to 2 features resulted only in a minimal loss of information as there exist string linear relationships between the original feature values


### Reducing from 4 to 1 dimension

The previous result shows that even only the first principal component explains over 92% of the variance. Depending on the threshold we set (e.g., 90%) we can even use PCA to reduce the data from 4 to only 1 feature; the steps are naturally exactly the same.

In [ ]:
pca_1d = PCA(n_components=1).fit(X)

X_pca_1d = pca_1d.transform(X)

Of course, now the plot is 1-dimensional, making it a bit less clear how well the classes are separated.

In [ ]:
plt.figure()
plt.scatter(X_pca_1d, [0]*X_pca_1d.shape[0], c=y, cmap=cm.tab10, s=50)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

With on feature remaining, the bar plot for visualizing the explained variances becomes a bit boring -- and technically, we already know the figure from the previous plot.

In [ ]:
plot_explained_variances(pca_1d.explained_variance_ratio_)

-------------------------------

## Linear Discriminant Analysis (LDA)

The general LDA approach is very similar to a Principal Component Analysis, but in addition to finding the component axes that maximize the variance of our data (PCA), we are additionally interested in the axes that maximize the separation between multiple classes (LDA).

In contrast to PCA, LDA is “supervised” and computes the directions ("linear discriminants") that will represent the axes that maximize the separation between multiple classes. Although it might sound intuitive that LDA is superior to PCA for a multi-class classification task where the class labels are known, this might not always be the case. For example, comparisons between classification accuracies for image recognition after using PCA or LDA show that PCA tends to outperform LDA if the number of samples per class is relatively small (PCA vs. LDA, A.M. Martinez et al., 2001). In practice, it is also not uncommon to use both LDA and PCA in combination: E.g., PCA for dimensionality reduction followed by an LDA.

It should be mentioned that LDA assumes normal distributed data, features that are statistically independent, and identical covariance matrices for every class. However, this only applies for LDA as classifier and LDA for dimensionality reduction can also work reasonably well if those assumptions are violated.


### Reducing from 4 to 2 dimensions

As for PCA we first reduce the IRIS dataset to 2 features. `scitkit-learn` comes also with an implementation of LDA and its usage is very similar to the one of PCA. However, note that the `fit()` method also requires the class labels `y` as input parameters.

In [ ]:
lda_2d = LinearDiscriminantAnalysis(n_components=2).fit(X, y)

X_lda_2d = lda_2d.transform(X)

Now that we have reduced the data to 2 features, we can visualize using a scatter plot.

In [ ]:
plt.figure()
plt.scatter(X_lda_2d[:, 0], X_lda_2d[:, 1], c=y, cmap=cm.tab10, s=25)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

Simply eyeballing the plot and comparing it with the one for PCA, one could argue that the result in terms of the separation of the data points is very similar. However, when looking at the explained variances again, the numbers are a bit different.

In [ ]:
print('Explained variances (normalized): {}'.format(lda_2d.explained_variance_ratio_))

print('Overall explained variance (normalized): {}'.format(sum(lda_2d.explained_variance_ratio_)))

...and the same numbers as a bar plot.

In [ ]:
plot_explained_variances(lda_2d.explained_variance_ratio_)

For this dataset, LDA provides a better reduction result, but note that this does not always have to be the case.



### Reducing from 4 to 1 dimension

With this result for the explained variance, we can make an even stronger case to reduce the number of features to only 1.

In [ ]:
lda_1d = LinearDiscriminantAnalysis(n_components=1).fit(X, y)

X_lda_1d = lda_1d.transform(X)

Again, just by looking at the plots, the results for PCA and LDA are very similar -- and in fact they are rather similar.

In [ ]:
plt.figure()
plt.scatter(X_lda_1d, [0]*X_lda_1d.shape[0], c=y, cmap=cm.tab10, s=50)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

We can skip the bar plot showing the explained variances since we already know that the number is 99% for the one feature.

----------------------------

## t-distributed Stochastic Neighbor Embedding (t-SNE)

t-SNE (t-Distributed Stochastic Neighbor Embedding) is a dimensionality reduction technique commonly used in data mining and visualization. It is particularly effective in preserving the local structure and capturing non-linear relationships in high-dimensional data.

Unlike PCA, which focuses on preserving global structure and capturing the maximum variance, t-SNE emphasizes the preservation of the proximity and similarity relationships between data points. It achieves this by creating a lower-dimensional representation of the data in such a way that similar data points are modeled to be close together, while dissimilar points are modeled to be far apart.

The main steps involved in t-SNE are as follows:

* **Compute similarities:** t-SNE starts by calculating pairwise similarities between data points. These similarities are typically based on a Gaussian kernel function, which assigns higher similarity to nearby points and lower similarity to distant points.

* **Construct the similarity matrix:** Using the computed similarities, a similarity matrix is constructed, representing the relationships between all pairs of data points.

* **Initialize embeddings:** The t-SNE algorithm begins with an initialization step where the embeddings (coordinates) for each data point in the lower-dimensional space are randomly initialized.

* **Define a cost function:** t-SNE defines a cost function that measures the similarity between the pairwise relationships in the original high-dimensional space and the lower-dimensional space. The algorithm aims to minimize this cost function.

* **Optimize the embeddings:** The algorithm iteratively updates the embeddings to minimize the cost function. It uses gradient descent techniques to find a configuration of the embeddings that best approximates the pairwise relationships in the high-dimensional space.

* **Repeat the optimization process:** The optimization process is repeated until convergence is achieved or a maximum number of iterations is reached.

t-SNE produces a lower-dimensional representation of the data, typically in two or three dimensions, which can be visualized in a scatter plot. In the lower-dimensional space, similar data points tend to cluster together, allowing for visual identification of clusters or groups within the data. It is particularly useful for visualizing high-dimensional datasets, exploring patterns, identifying clusters, and gaining insights into the underlying structure of the data.

It is important to note that t-SNE is computationally intensive, and the interpretation of the results should be done with caution, as the distances and relationships between points in the lower-dimensional space may not be directly interpretable. t-SNE is primarily a visualization tool and is often used as an exploratory analysis technique in combination with other methods.

Some things to consider when using t-SNE:

* For the algorithm to execute properly, the perplexity should be smaller than the number of points. The suggested perplexity is 5..50
* Sometimes, different runs with the same hyper parameters may produce different results.
* Cluster sizes in any t-SNE plot must not be evaluated for standard deviation, dispersion or any other similar measures. This is because t-SNE expands denser clusters and contracts sparser clusters to even out cluster sizes. This is one of the reasons for the crisp and clear plots it produces.
* Distances between clusters may change because global geometry is closely related to optimal perplexity. And in a dataset with many clusters with different numbers of elements one perplexity cannot optimize distances for all clusters.
* Patterns may be found in random noise as well, so multiple runs of the algorithm with different sets of hyperparameters must be checked before deciding if a pattern exists in the data.
* Different cluster shapes may be observed at different perplexity levels.
* Topology cannot be analyzed based on a single t-SNE plot, multiple plots must be observed before making any assessment.


### Reducing from 4 to 2 dimensions

We again use `scikit-learn`'s implementation of t-SNE, starting with reducing the IRIS dataset to 2 features. You can try different different values for `perplexity` to see how it affects the results. You can also run `TSNE` with the same parameters multiple times to see how the result changes as t-SNE is not deterministic, depending on the initialization of the lower-dimensional data values.

In [ ]:
X_tsne_2d = TSNE(n_components=2, perplexity=50).fit_transform(X)

Visualizing the result as scatter plot:

In [ ]:
plt.figure()
plt.scatter(X_tsne_2d[:, 0], X_tsne_2d[:, 1], c=y, cmap=cm.tab10, s=25)

plt.tick_params(labelsize=14)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

This plot now looks noticeably different compared to the ones for PCA and LDA (both based on linear transformations), as t-SNE uses a very different approach to reduce the dimensionality of the data.


### Reducing from 4 to 1 dimension

For the sake of completeness, we can finally apply t-SNE on the IRIS dataset to reduce it to only one feature. Again, try different values for `perplexity` and multiple runs for the same parameter values to see how the results change.

In [ ]:
X_tsne_1d = TSNE(n_components=1, perplexity=50).fit_transform(X)

We can visualize the dataset using a 1-dim scatter plot:

In [ ]:
plt.figure()
plt.scatter(X_tsne_1d, [0]*X_tsne_1d.shape[0], c=y, cmap=cm.tab10, s=50)
plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)  
plt.tight_layout()
plt.show()

The plot can vary noticeable for different runs, even with the same parameter values.

---

## Summary

**PCA (Principal Component Analysis)** is a widely used dimensionality reduction technique that aims to transform a dataset into a lower-dimensional space while preserving the maximum amount of variation. It is effective in capturing global patterns and reducing computational complexity. PCA provides a linear transformation and is suitable for linearly correlated data. Its strengths include simplicity, computational efficiency, and interpretability. However, PCA may not capture complex non-linear relationships and can be sensitive to outliers. It also does not consider class labels, making it less suitable for tasks involving supervised learning.

**LDA (Linear Discriminant Analysis)**, on the other hand, is a dimensionality reduction technique that specifically focuses on class separation. It aims to find a lower-dimensional space that maximizes the between-class variance and minimizes the within-class variance. LDA is particularly useful for classification tasks as it considers class labels. It can capture both global and local structures in the data. However, LDA assumes linearity and requires the number of samples to be greater than the number of features. LDA can struggle with high-dimensional data and may not perform well if the classes overlap significantly.

**t-SNE (t-Distributed Stochastic Neighbor Embedding)** is a non-linear dimensionality reduction technique designed to preserve local relationships and capture non-linear structures in the data. It is effective in visualizing high-dimensional data and identifying clusters or groups. t-SNE is especially useful for exploratory analysis and visualization. However, t-SNE can be computationally expensive and sensitive to the choice of hyperparameters. It does not provide a direct interpretation of the distances in the lower-dimensional space, making it less suitable for precise quantitative analysis.

In summary, PCA is a versatile technique that captures global patterns, LDA focuses on class separation for classification tasks, and t-SNE is effective in visualizing complex non-linear structures. The choice of technique depends on the specific goals, characteristics of the data, and the trade-offs between interpretability, computational efficiency, and capturing the desired relationships in the data. It is often beneficial to combine multiple dimensionality reduction techniques or use them in conjunction with other analysis methods for a comprehensive understanding of the data.
